### Библиотеки

In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
import pickle
from Experiment_2d_thermal import *

In [ ]:
import sys
# sys.path.append(fr'/home/mex0/Documents/TukhbatullinAL/diplom_2/Git_stencil-net/Git_stencil-net/')
sys.path.append('D:\myProjects\diplom_2\GIT_stencil-net\GIT_stencil-net')
sys.path.append('D:\myProjects\diplom_2\GIT_stencil-net\GIT_stencil-net/5_2d_thermal_conductivity/')

import funcs_2d_thermal as f2dt

In [ ]:
BC_TYPE='dirichlet'; BC_VALUES=[0,1]

### Описание эксперимента:

In [ ]:
"""
2д решение задачи теплопроводности
"""

### 0. Параметры для генерации данных

In [ ]:
folder='Experiment_2d_thermal_conductivity'

In [ ]:
# epochs_lst=[1,100,1_000,5_000,10_000]
# epochs_lst=[30_000]
epochs_lst=[100]

### 1. Генерация данных

In [ ]:
n=11
L=1
T=0.1
h=L/(n-1)

kurant=0.25
tau=np.round(kurant*(h**2),6)

x_lst=[i*h for i in range(n)]

v=np.zeros((n,n))
for i in range(0,n):
    for j in range(0,n):
        if i==n-1 or j==n-1:
            v[i][j]=BC_VALUES[-1]

# Конечн-разн схема
v_fact=np.array(f2dt.generate_data(True,v,T,L,kurant,h,n,CUSTOM_TAU=tau,save_flg=False)[0])

cmap='coolwarm'

plt.figure()
plt.imshow(v_fact[:,:,0],cmap=cmap)
plt.colorbar()

plt.figure()
plt.imshow(v_fact[:,:,-1],cmap=cmap)
plt.colorbar()

In [ ]:
1/h/h,4/h/h

In [ ]:
print('h',h,h**2)
print('tau',tau)
assert tau/h<=1,'не выполнено ограничение на число Куранта'

In [ ]:
v_fact.shape

### Чтение данных

In [ ]:
# path=fr'D:\myProjects\diplom_2\GIT_stencil-net\GIT_stencil-net\5_2d_thermal_conductivity'
# v_fact=pickle.load(open(fr'{path}/2d_advection_fact_2var.pkl','rb'))

In [ ]:
v_fact.shape

### Линеаризация

In [ ]:
def linearization(v_fact):

    n=len(v_fact)
    A=np.zeros((n*n))

    for i in range(n):
        for j in range(n):
            A[i+n*j]=v_fact[i,j]

    return A

def get_value(A,i,j):

    n=int(np.sqrt(len(A)))
    
    if i>=n and j>=n:
        value = A[i-n+n*j-n]
    elif i>=n:
        value = A[i-n+n*j]
    elif j>=n:
        value = A[i+(n*j)-n]
    else:
        value = A[i+n*j]

    return value

def linearization_inverse(v_fact):

    n=int(np.sqrt(len(v_fact)))
    v_fact_mat=np.zeros((n,n))

    for i in range(n):
        for j in range(n):
            v_fact_mat[i,j]=get_value(v_fact,i,j)


    return v_fact_mat

In [ ]:
v_fact.shape

In [ ]:
v_fact_linearizated=np.zeros((n*n,v_fact.shape[2]))
for j in range(v_fact.shape[2]):
    v_fact_linearizated[:,j]=linearization(v_fact[:,:,j])
v_fact_linearizated.shape

### 2. Сабсемплинг

In [ ]:
t_factor = 1  # time subsample factor 
s_factor = 1  # space subsample factor

# Tsim=len(v_fact[0])
# dxc,dtc,coarse_t,coarse_x,v_coarse,Lxc,\
# Ltc,v_coarse_train,v_coarse_test=subsampling(s_factor,t_factor,h,tau,Tsim,n,v_fact,train_split=train_split)

dxc=h
dtc=tau
# coarse_t,coarse_x = ...
v_coarse=np.copy(v_fact_linearizated)
Lxc = n*n
# Ltc=...
v_coarse_train = v_coarse[:,:int(v_coarse.shape[1]*train_split)]
v_coarse_test = v_coarse[:,int(v_coarse.shape[1]*train_split):]

In [ ]:
v_coarse_train.shape

In [ ]:
v_coarse_test.shape

In [ ]:
v_coarse.shape

In [ ]:
pickle.dump(v_fact,open(folder+'/v_fact.pkl','wb'))

### 3. Тренировка модели

In [ ]:
epochs_lst

In [ ]:
for epochs in epochs_lst:
    
    net, loss_lst,loss=train_net(MLPConv,v_coarse_train,epochs,dtc,
              fs,
              neurons,
              hidden_layers_num,
              lr,
              m,
              has_backward,
              method,
              decay_const,
              bc_type=BC_TYPE,bc_values=BC_VALUES,
              verbose=True,
              verbose_step=10)
    
    pickle.dump(net, open(fr'{folder}/models/net_epochs={epochs}_device={device}', 'wb'))
    

In [ ]:
lr

In [ ]:
has_backward

In [ ]:
net.layer[0].weight

In [ ]:
assert 1==2, 'stop point'

### 4. Симуляция прогноза моделью

In [ ]:
NN_sim_lst=[]

weights=[]
biases=[]

for epochs in epochs_lst:
    
    net = pickle.load(open(fr'{folder}/models/net_epochs={epochs}_device={device}', 'rb'))
    NN_sim,T_sim,x_sim=make_simulation(net,v_coarse,L,Lxc,dtc,method=method,
                                              bc_type=BC_TYPE,bc_values=BC_VALUES)
    
    NN_sim_lst.append(NN_sim)

    try:
        weights.append(net.layer[0].weight.data.numpy())
    except:
        weights.append(net.layer[0].weight.cpu().data.numpy())
    biases.append(None)

In [ ]:
net.layer[0].weight

In [ ]:
pd.DataFrame({'epoch':epochs_lst,#+[['finite_diff_methods']],
              'coefs':weights,#+[[1/((dxc)),-1/(dxc),0]],
              'bias':biases})#+[[None]]})

### 5. Результаты

In [ ]:
# err=np.abs(v_coarse-NN_sim)
# mae_list=[err[:,i].mean() for i in range(err.shape[1])]
# mae_df=pd.DataFrame({'method':method,'mae_list':mae_list})#.to_csv(fr'{method}_mae.csv')

In [ ]:

for i in range(len(epochs_lst)):
    
    print(fr'---------------Epochs = {epochs_lst[i]}-------------------')

    make_subplot_graphs_2d(NN_sim, v_coarse, x_sim, T_sim, T, dtc, 11,
                       figsize=[9,2],
                       vmin_diff=-1,
                       vmax_diff=1,
                       save_flg=True,
                       save_path=folder+'/graphs/',
                       save_name=fr'SUBPLOT_by_time_{epochs_lst[i]}'
                       )
    
    err=metric_by_time(NN_sim,v_coarse,T,dtc,ymax=None,
                save_flg=True,
                save_path=folder+'/graphs/',
                save_name=fr'MAE_by_time_{epochs_lst[i]}'
                )

    print('\n')
print('done')

In [ ]:
assert 1==2,'stop point'

### cpu_vs_cuda

In [ ]:
model_cpu=pickle.load(open(fr'{folder}/models/net_epochs={epochs}_device=cpu', 'rb'))
model_cuda=pickle.load(open(fr'{folder}/models/net_epochs={epochs}_device=cuda', 'rb'))

In [ ]:
model_cpu.layer[0].weight.data.numpy()

In [ ]:
model_cuda.layer[0].weight.cpu().data.numpy()

### RK3_mae vs E1_mae

In [ ]:
rk3_mae=pd.read_csv(fr'RK3_mae.csv')
e1_mae=pd.read_csv(fr'E1_mae.csv')
plt.plot(rk3_mae['mae_list'],label='RK3')
plt.plot(e1_mae['mae_list'],label='E1')
plt.grid()
plt.legend()